### [연속 펄스 부분 수열의 합](https://school.programmers.co.kr/learn/courses/30/lessons/161988?language=python3)

- 부분 수열의 최대값은 누적합 간의 차이의 최대값으로 측정이 가능하며
- 아래와 같은 펄스가 적용될 경우 최대값과 최소값 간의 차이(크기)가 그 값이 됨

In [ ]:
def solution(sequence):
    prefix_sums = [0]
    curr_sum = 0
    
    for i, val in enumerate(sequence):
        if i % 2 == 0:
            curr_sum += val
        else:
            curr_sum -= val
        prefix_sums.append(curr_sum)

    return max(prefix_sums) - min(prefix_sums)


### [징검다리 건너기](https://school.programmers.co.kr/learn/courses/30/lessons/64062)

- 처음 생각한 아래와 같은 코드로 정합성은 맞추지만 효율성을 맞추지 못함

In [ ]:
def solution(stones, k):
    # 기존 코드처럼 {인덱스: [남은횟수, 점프거리]} 구조 유지
    # 시작 지점을 위해 -1 인덱스 추가 (개울 왼쪽)
    data = {i: [stones[i], 1] for i in range(len(stones))}
    data[-1] = [float('inf'), 1] 
    
    answer = 0
    while True:
        curr = -1 # 시작 위치
        
        # 한 명의 친구가 건너는 과정
        while curr < len(stones):
            jump = data[curr][1]
            if jump > k: # 점프할 거리가 k보다 크면 종료
                return answer
            
            curr += jump # 다음 돌로 이동
            
            if curr < len(stones):
                data[curr][0] -= 1 # 돌 밟음
                
                if data[curr][0] == 0: # 돌이 방금 0이 되었다면 거리 갱신
                    # 현재 돌(curr)을 목적지로 하던 이전 돌을 찾아 거리 합산
                    prev = curr - 1
                    while prev >= -1:
                        if prev in data and data[prev][0] > 0:
                            # 이전 살아있는 돌의 점프 거리를 현재 돌의 점프 거리만큼 추가
                            data[prev][1] += data[curr][1]
                            break
                        prev -= 1
        answer += 1

- 이분 탐색으로 해결

In [ ]:
def solution(stones, k):
    # 건너갈 수 있는 친구의 수 범위를 이분 탐색으로 설정
    left = 1
    right = 200000000 # 제한 사항의 최대값
    answer = 0
    
    while left <= right:
        mid = (left + right) // 2 # mid명의 친구가 건널 수 있는지 확인
        
        # 연속으로 0이 되는(못 밟는) 돌의 개수 체크
        cnt = 0
        max_cnt = 0
        for s in stones:
            if s - mid < 0: # mid번째 친구가 밟기 전에 이미 0 이하가 됨
                cnt += 1
            else:
                cnt = 0
            
            if cnt > max_cnt:
                max_cnt = cnt
        
        # 연속된 0의 개수가 k이상이면 mid명은 못 건너감
        if max_cnt < k:
            answer = mid
            left = mid + 1
        else:
            right = mid - 1
            
    return answer